<a href="https://colab.research.google.com/github/Lisaaa2021/NLP-practice/blob/main/sentiment_analysis_with_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise objectives:
- Convert words to vectors with Word2Vec
- Use the word representation given by Word2vec to feed a RNN
- Use transfer learning for word2vec

In [2]:
#load the data

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
  
        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]
    
    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=10)

In [3]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=X_train)
#this word2vec contains all the sentences in X_train and its vectors with size == default 100

In [4]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence: #for each word in one sentence
        if word in word2vec.wv: 
            embedded_sentence.append(word2vec.wv[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences: #for sentence in 2500 sentences
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed #has all words in the X_train

# Embed the training and test sentences
X_train_embed = embedding(word2vec, X_train)
X_test_embed = embedding(word2vec, X_test)


# Pad the training and test embedded sentences
X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=200)
X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=200)

# Baseline model

In [33]:
from sklearn.metrics import accuracy_score

unique, counts = np.unique(y_train, return_counts=True)
counts = dict(zip(unique, counts))
print('Number of labels in train set', counts)

y_pred = 0 if counts[0] > counts[1] else 1

print('Baseline accuracy: ', accuracy_score(y_test, [y_pred]*len(y_test)))

Number of labels in train set {0: 1265, 1: 1235}
Baseline accuracy:  0.492


# The model

In [34]:
from tensorflow.keras import layers, Sequential
def init_model():
  model = Sequential()
  model.add(layers.Masking())
  model.add(layers.LSTM(20, activation='tanh'))
  model.add(layers.Dense(15, activation='relu'))
  model.add(layers.Dense(1, activation='sigmoid'))

  model.compile(loss='binary_crossentropy',
                optimizer='rmsprop',
                metrics=['accuracy'])
  return model

In [48]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(patience = 5, restore_best_weights=True)

model = init_model()

history = model.fit(X_train_pad, y_train, 
          batch_size=16, 
          epochs=1000, 
          validation_split=0.3,
          callbacks=[es])


Epoch 1/1000
110/110 [==============================] - 20s 130ms/step - loss: 0.6925 - accuracy: 0.5194 - val_loss: 0.6823 - val_accuracy: 0.5893
Epoch 2/1000
110/110 [==============================] - 13s 118ms/step - loss: 0.6791 - accuracy: 0.5680 - val_loss: 0.6614 - val_accuracy: 0.6253
Epoch 3/1000
110/110 [==============================] - 13s 118ms/step - loss: 0.6603 - accuracy: 0.6126 - val_loss: 0.6558 - val_accuracy: 0.6333
Epoch 4/1000
110/110 [==============================] - 13s 119ms/step - loss: 0.6541 - accuracy: 0.6143 - val_loss: 0.6526 - val_accuracy: 0.6347
Epoch 5/1000
110/110 [==============================] - 13s 119ms/step - loss: 0.6497 - accuracy: 0.6189 - val_loss: 0.6455 - val_accuracy: 0.6187
Epoch 6/1000
110/110 [==============================] - 13s 118ms/step - loss: 0.6340 - accuracy: 0.6411 - val_loss: 0.6439 - val_accuracy: 0.6413
Epoch 7/1000
110/110 [==============================] - 13s 118ms/step - loss: 0.6264 - accuracy: 0.6543 - val_loss: 0

# Evaluate  the model

In [40]:
model.evaluate(X_test_pad,y_test)

79/79 [==============================] - 2s 29ms/step - loss: 0.6852 - accuracy: 0.5920


[0.6851630806922913, 0.5920000076293945]

# Trained Word2Vec - Transfer Learning

In [36]:
import gensim.downloader as api
print(list(api.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [37]:
word2vec_transfer = api.load("glove-wiki-gigaword-50")

In [41]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence_with_TF(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec:
            embedded_sentence.append(word2vec[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence_with_TF(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

In [42]:
# Embed the training and test sentences
X_train_embed_2 = embedding(word2vec_transfer, X_train)
X_test_embed_2 = embedding(word2vec_transfer, X_test)

#Padding
X_train_pad_2 = pad_sequences(X_train_embed_2, dtype='float32', padding='post', maxlen=200)
X_test_pad_2 = pad_sequences(X_test_embed_2, dtype='float32', padding='post', maxlen=200)

In [43]:
X_train_pad.shape, X_train_pad_2.shape

((2500, 200, 100), (2500, 200, 50))

In [46]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=5, restore_best_weights=True)

model = init_model()
 
model.fit(X_train_pad_2, y_train, 
          batch_size = 16,
          epochs=1000,
          validation_split=0.3,
          callbacks=[es]
         )

Epoch 1/1000
110/110 [==============================] - 18s 122ms/step - loss: 0.6955 - accuracy: 0.5114 - val_loss: 0.6876 - val_accuracy: 0.5347
Epoch 2/1000
110/110 [==============================] - 12s 111ms/step - loss: 0.6797 - accuracy: 0.5771 - val_loss: 0.6709 - val_accuracy: 0.5747
Epoch 3/1000
110/110 [==============================] - 12s 112ms/step - loss: 0.6632 - accuracy: 0.6029 - val_loss: 0.6585 - val_accuracy: 0.6107
Epoch 4/1000
110/110 [==============================] - 12s 112ms/step - loss: 0.6436 - accuracy: 0.6246 - val_loss: 0.6345 - val_accuracy: 0.6400
Epoch 5/1000
110/110 [==============================] - 12s 113ms/step - loss: 0.6095 - accuracy: 0.6686 - val_loss: 0.5957 - val_accuracy: 0.6880
Epoch 6/1000
110/110 [==============================] - 12s 112ms/step - loss: 0.5858 - accuracy: 0.6960 - val_loss: 0.5983 - val_accuracy: 0.6893
Epoch 7/1000
110/110 [==============================] - 12s 112ms/step - loss: 0.5657 - accuracy: 0.7137 - val_loss: 0

In [47]:
res = model.evaluate(X_test_pad_2, y_test, verbose=0)

print(f'The accuracy evaluated on the test set is of {res[1]*100:.3f}%')

The accuracy evaluated on the test set is of 73.400%
